In [7]:
# %pip install kagglehub

In [44]:
# import kagglehub

# # Descargar el dataset
# path = kagglehub.dataset_download("stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset")

# print("Path to dataset files:", path)

In [45]:
# import kagglehub
# import shutil
# import os

# # Descargar el dataset
# path = kagglehub.dataset_download("stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset")

# # Ruta personalizada
# custom_path = "dataEx"

# # Crear el directorio si no existe
# os.makedirs(custom_path, exist_ok=True)

# # Mover archivos descargados al nuevo directorio
# for file in os.listdir(path):
#     shutil.move(os.path.join(path, file), os.path.join(custom_path, file))

# print(f"Archivos movidos a: {custom_path}")

# Examen Bimestral - Diseño de un Sistema Básico de Recuperación de Información

## Requerimientos
### 1. Preprocesamiento de Datos

In [ ]:
# %pip install TfidfVectorizer
# %pip install pandas nltk scikit-learn

In [47]:
import pandas as pd

# Cargar el dataset de películas
movies = pd.read_csv('dataEx/rotten_tomatoes_movies.csv')
critic_review = pd.read_csv('dataEx/rotten_tomatoes_critic_reviews.csv')

# Visualizar las primeras filas para confirmar las columnas
# print(movies.head())
# print(critic_review.head())
print(movies[['movie_title', 'critics_consensus']].head())

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                   critics_consensus  
0  Though it may seem like just another Harry Pot...  
1  Nicole Holofcener's newest might seem slight i...  
2  Blake Edwards' bawdy comedy may not score a pe...  
3  Sidney Lumet's feature debut is a superbly wri...  
4  One of Disney's finest live-action adventures,...  


In [20]:
import re
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, download
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer

In [64]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [65]:
import os
import nltk

nltk_data_path = os.path.expanduser('~') + '\\nltk_data'
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)

nltk.data.path.append(nltk_data_path)

# Descargar recursos
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('averaged_perceptron_tagger', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to C:\Users\USER\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\USER\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [68]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
# Función para convertir etiquetas POS de nltk a WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Por defecto

# Función de preprocesamiento
def preprocesar_contenido(contenido, stopwords, signos_puntuacion):
    # Validar contenido
    if contenido is None or not isinstance(contenido, str):
        return ''
    
    # Definir el lematizador
    lemmatizer = WordNetLemmatizer()

    # Convertir texto a minúsculas
    contenido = contenido.lower()

    # Reemplazar caracteres especiales y números
    contenido = re.sub(r'\d+', '', contenido)  # Eliminar números
    contenido = re.sub(r'[^\w\s]', '', contenido)  # Eliminar puntuación
    contenido = re.sub(r'\s+', ' ', contenido)  # Eliminar espacios redundantes

    # Tokenización
    contenido_tokenizado = word_tokenize(contenido)

    # Eliminar stopwords
    tokens_sin_stopwords = [word for word in contenido_tokenizado if word not in stopwords]

    # Etiquetado POS y lematización
    tokens_pos = pos_tag(tokens_sin_stopwords)
    tokens_lematizados = [
        lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in tokens_pos
    ]

    # Reconstruir texto lematizado
    return ' '.join(tokens_lematizados)

# Función para obtener stopwords
def obtener_stopwords(idioma='english'):
    return set(stopwords.words(idioma))

# Preprocesar dataset
def preprocesar_dataset(dataframe, columna_texto):
    # Obtener stopwords
    stop_words = obtener_stopwords('english')

    # Aplicar preprocesamiento a la columna especificada
    dataframe['processed_text'] = dataframe[columna_texto].fillna('').apply(
        lambda x: preprocesar_contenido(x, stop_words, [])
    )
    return dataframe

# Preprocesar archivos
def cargar_y_preprocesar(ruta, columna_texto):
    try:
        # Cargar archivo
        dataframe = pd.read_csv(ruta)
        print(f"Columnas disponibles en el dataset: {dataframe.columns}")
        
        # Preprocesar si la columna existe
        if columna_texto in dataframe.columns:
            dataframe = preprocesar_dataset(dataframe, columna_texto)
            print(dataframe[[columna_texto, 'processed_text']].head())
        else:
            print(f"La columna '{columna_texto}' no está disponible en el dataset.")
        
        return dataframe
    except FileNotFoundError:
        print(f"El archivo '{ruta}' no fue encontrado. Verifica la ruta.")
        return None
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

# Cargar y procesar datasets
movies = cargar_y_preprocesar('dataEx/rotten_tomatoes_movies.csv', 'critics_consensus')
critic_reviews = cargar_y_preprocesar('dataEx/rotten_tomatoes_critic_reviews.csv', 'review')

Columnas disponibles en el dataset: Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')
                                   critics_consensus  \
0  Though it may seem like just another Harry Pot...   
1  Nicole Holofcener's newest might seem slight i...   
2  Blake Edwards' bawdy comedy may not score a pe...   
3  Sidney Lumet's feature debut is a superbly wri...   
4  One of Disney's finest live-action adventures,...   

                                      processed_text  
0  though may seem like another harry potter knoc...  


### 2. Construcción del Sistema

### 3. Simulación de Consultas: El sistema debe mostrar un listado de las películas más relevantes para cada consulta, ordenadas según su relevancia.

### 4. Análisis de Resultados